In [1]:
   import tensorflow as tf
   print(f"TF version: {tf.__version__}")
   gpus = tf.config.list_physical_devices('GPU')
   if gpus:
       print("GPU is available")
       print("Num GPUs Available: ", len(gpus))
   else:
       print("No GPUs available in your system.")

2025-06-11 00:36:21.621042: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-11 00:36:21.675513: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749569781.706570   20772 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749569781.716500   20772 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749569781.764497   20772 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

TF version: 2.19.0
GPU is available
Num GPUs Available:  1


In [10]:
import pandas as pd
import numpy as np
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

df = pd.read_csv('~/Food_Delivery_Times.csv')

df = df.dropna()

X = df.drop(['Order_ID', 'Delivery_Time_min'], axis='columns')
y = df['Delivery_Time_min']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(f"훈련 데이터 수: {len(X_train)}개, 테스트 데이터 수: {len(X_test)}개")

categorical_transformer = OneHotEncoder(handle_unknown='ignore')

numeric_transformer = MinMaxScaler()

preprocessor = ColumnTransformer(
    transformers=[
        ('Traffic_Level', categorical_transformer, ['Traffic_Level']),
        ('Time_of_Day', categorical_transformer, ['Time_of_Day']),
        ('Vehicle_Type', categorical_transformer, ['Vehicle_Type']),
        ('Weather', categorical_transformer, ['Weather']),
        ('Distance_km', numeric_transformer, ['Distance_km']),
        ('Preparation', numeric_transformer, ['Preparation_Time_min']),
        ('Courier', numeric_transformer, ['Courier_Experience_yrs']),
    ],
    remainder='passthrough' 
)


model = Sequential()

model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])


pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', model)])

early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

history = pipeline.fit(X_train, y_train,
                       regressor__epochs=2000,
                       regressor__batch_size=1,
                       regressor__validation_split=0.1,
                       regressor__callbacks=[early_stopping],
                       regressor__verbose=1)


model.summary()

# Preprocess X_test before evaluating the model
X_test_processed = pipeline.named_steps['preprocessor'].transform(X_test)

loss, mae = model.evaluate(X_test_processed, y_test, verbose=0)
print(f"\n{loss}")
print(f"\n테스트 데이터에 대한 최종 Mean Absolute Error (MAE): {mae:,.2f}분")
print(f"-> 모델의 예측치가 실제값과 평균적으로 {mae:,.2f}분 정도 차이남을 의미합니다.")


predictions = pipeline.predict(X_test)

results_df = pd.DataFrame({
    '실제 시간': y_test.values.flatten(),
    '예측 시간': predictions.flatten()
})
results_df['차이'] = results_df['실제 시간'] - results_df['예측 시간']

print("\n--- 실제 시간과 예측 시간 비교 (상위 5개) ---")
print(results_df.head())


훈련 데이터 수: 706개, 테스트 데이터 수: 177개
Epoch 1/2000
635/635 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 38.9950 - mae: 38.9950 - val_loss: 11.4894 - val_mae: 11.4894
Epoch 2/2000
635/635 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 11.3937 - mae: 11.3937 - val_loss: 8.3942 - val_mae: 8.3942
Epoch 3/2000
635/635 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 8.1879 - mae: 8.1879 - val_loss: 7.4469 - val_mae: 7.4469
Epoch 4/2000
635/635 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 6.5561 - mae: 6.5561 - val_loss: 7.3084 - val_mae: 7.3084
Epoch 5/2000
635/635 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 6.2040 - mae: 6.2040 - val_loss: 7.2819 - val_mae: 7.2819
Epoch 6/2000
635/635 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 5.8879 - mae: 5.8879 - val_loss: 7.7055 - val_mae: 7.7055
Epoch 7/2000
635/635 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 6.4534 - mae: 6.4534 - val_loss: 7.7991 - val_mae: 7.7991
Epoch 8/2000
635/635 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 6.5194 - mae: 6.5194 - val_loss: 7.0731 - val_mae: 7.0731
Ep

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_28 (Dense)                │ (1, 32)                │           608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (1, 16)                │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_30 (Dense)                │ (1, 8)                 │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (1, 1)                 │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,845 (15.02 KB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,564 (10.02 KB)


6.301714897155762

테스트 데이터에 대한 최종 Mean Absolute Error (MAE): 6.30분
-> 모델의 예측치가 실제값과 평균적으로 6.30분 정도 차이남을 의미합니다.
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step

--- 실제 시간과 예측 시간 비교 (상위 5개) ---
   실제 시간      예측 시간        차이
0     36  35.057354  0.942646
1     77  86.150002 -9.150002
2     61  52.226643  8.773357
3     39  47.849434 -8.849434
4     57  54.530689  2.469311


/home/minux/miniforge3/envs/jupyter_env/lib/python3.11/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
